In [85]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

onlyfiles = [f for f in listdir("Guinea") if isfile(join("Guinea", f))]
#print(onlyfiles)
list_files = pd.DataFrame(onlyfiles)
#print(list_files.columns.tolist())
list_files=list_files.rename(columns={0: 'Sample'})
#print(list_files)
list_files.Sample= list_files.Sample.str.split("_").str[0]
list_files=list_files.drop_duplicates()
#print(list_files)

Guinea_df = pd.read_excel (r'NMS results_for Guinea TES 2017-2019_Final 03-08-2021_Final Call.xlsx',  sheet_name='Recurrent samples')
Guinea_master_df = pd.read_excel (r'Guinea Paired Masterfile.xlsx', sheet_name='MS to NGS Sample Naming ')
Guinea_master_df=Guinea_master_df.reset_index(drop=True)
list_files=list_files.reset_index(drop=True)

Guinea_master_df=Guinea_master_df.rename(columns={"AMD_ID": 'Sample'})
#rint(Guinea_master_df)
#Guinea_master_df=Guinea_master_df.sort_index(inplace=True)
#print(list_files['Sample'])
#print(Guinea_master_df['AMD_ID'])
#list_files['C'] = np.where(list_files['Sample'] == Guinea_master_df['AMD_ID'], Guinea_master_df['AMD_ID'], np.nan)
#print(Guinea_master_df)
list_files=list_files.merge(Guinea_master_df, how='inner', on='Sample')
list_files = list_files[['CSID', 'Sample']]
#list_files.to_csv('out.csv')
Guinea_df = Guinea_df.iloc[2: , :]
new_header = Guinea_df.iloc[0] #grab the first row for the header
Guinea_df = Guinea_df[1:] #take the data less the header row
Guinea_df.columns = new_header #set the header row as the df header
#print(Guinea_df)
#Guinea_df = Guinea_df.rename(columns={np.nan: 'Rec/Rei'})
Guinea_df.columns = Guinea_df.columns.fillna('Rec/Rei')
#print(Guinea_df)
#Guinea_df.to_csv('out2.csv')
list_files=list_files.merge(Guinea_df, how='inner', on='CSID')
list_files = list_files[['CSID', 'Sample', 'Day 0 or failure', 'Treatment arm','Rec/Rei']]
list_files['Rec/Rei2'] = np.where(list_files['Rec/Rei']>0.5, 'Recrudescence', 'Reinfection')
list_files=list_files.drop(columns=['Rec/Rei'])
list_files=list_files.rename(columns={"Rec/Rei2": 'Molecular classification'})
list_files=list_files.rename(columns={"Sample": 'AMD_ID'})

list_files.to_csv('Guinea_NCBI.csv')

#Guinea_df_merged = pd.merge(Guinea_df, Guinea_master_df, on='CSID')

#print(Guinea_df_merged)